<a href="https://colab.research.google.com/github/Mahnazshamissa/Asigmo-Hackathon-/blob/main/Humor_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd '/content/gdrive/My Drive/data'

/content/gdrive/My Drive/data


In [ ]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Loading data from CSV file
data = pd.read_csv('hackathon_train.csv', sep=',', encoding="UTF-8")
data.head()

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1


In [ ]:
# Check for the null values
print(data.isnull().any(axis = 0))

id                   False
text                 False
is_humor             False
humor_rating          True
humor_controversy     True
offense_rating       False
dtype: bool


In [ ]:
# Check the first row of text
data['text'].iloc[0]

"TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit..."

In [ ]:
# Replace special symbols and digits in text column
data['text'] = data['text'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [ ]:
# Get features and labels
features = data['text']
labels = data['is_humor']

In [ ]:
# Stemming the data
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

In [ ]:
# Vectorize data with maximum of 5000 features
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()
features.shape

(8000, 5000)

In [ ]:
# Get training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.2, random_state = 0)

In [ ]:
# Use Linear Support Vector Classifier
lsvc = LinearSVC()

# Train the model
lsvc.fit(features_train, labels_train)

# Get the mean F1 score of train and test data
print(f1_score(labels_train, lsvc.predict(features_train)))
print(f1_score(labels_test, lsvc.predict(features_test)))

0.9806598407281
0.8859604662949823


In [ ]:
# Use Gaussian Naive Bayes
gnb = GaussianNB()

# Train the model
gnb.fit(features_train, labels_train)

# Get the mean F1 score of train and test data
print(f1_score(labels_train, gnb.predict(features_train)))
print(f1_score(labels_test, gnb.predict(features_test)))

0.8600722021660651
0.7083576855639977


In [ ]:
# Use Logistic Regression
lr = LogisticRegression()

# Train the model
lr.fit(features_train, labels_train)

# Get the mean F1 score of train and test data
print(f1_score(labels_train, lr.predict(features_train)))
print(f1_score(labels_test, lr.predict(features_test)))

0.925765274507374
0.884272997032641


In [ ]:
# Use Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)

# Train the model
rfc.fit(features_train, labels_train)

# Get the mean F1 score of train and test data
print(f1_score(labels_train, rfc.predict(features_train)))
print(f1_score(labels_test, rfc.predict(features_test)))

0.9949161159125571
0.8261316872427984


In [ ]:
from sklearn.linear_model import RidgeClassifier

# Use Ridge Classifier
rdg = RidgeClassifier()

# Train the model
rdg.fit(features_train, labels_train)

# Get the mean F1 score of train and test data
print(f1_score(labels_train, rdg.predict(features_train)))
print(f1_score(labels_test, rdg.predict(features_test)))

0.9685455460493206
0.8866599799398194


In [ ]:
!pip install pycaret

     |████████████████████████████████| 256kB 6.3MB/s 
     |████████████████████████████████| 266kB 8.1MB/s 
     |████████████████████████████████| 6.8MB 9.3MB/s 
     |████████████████████████████████| 148.9MB 84kB/s 
     |████████████████████████████████| 276kB 39.8MB/s 
     |████████████████████████████████| 13.9MB 38.1MB/s 
     |████████████████████████████████| 66.3MB 95kB/s 
     |████████████████████████████████| 1.6MB 40.9MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 1.8MB 32.1MB/s 
     |████████████████████████████████| 174kB 44.7MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 3.1MB 34.0MB/s 
     |████████████████████████████████| 604kB 37.1MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 1.1MB 27.2MB/s 
     |████████████████████████████████| 153kB 44.5MB/s 
 

In [ ]:
from pycaret.classification import *

In [ ]:
df = pd.DataFrame(np.array(features))
df['labels'] = labels
df.columns = ["C"+str(i) for i in range(1, df.shape[1]+1)]

In [ ]:
# setup the dataset
grid = setup(data=df, target=df.columns[-1], html=False, silent=True, verbose=False)

In [ ]:
# evaluate models and compare models
best = compare_models(sort = 'F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.64,6.547


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
nb,Naive Bayes,0.6971,0.7218,0.6373,0.8397,0.7243,0.4018,0.4212,0.521


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
dt,Decision Tree Classifier,0.7296,0.7117,0.7834,0.7841,0.7833,0.4234,0.4241,2.384
nb,Naive Bayes,0.6971,0.7218,0.6373,0.8397,0.7243,0.4018,0.4212,0.521


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
dt,Decision Tree Classifier,0.7296,0.7117,0.7834,0.7841,0.7833,0.4234,0.4241,2.384
nb,Naive Bayes,0.6971,0.7218,0.6373,0.8397,0.7243,0.4018,0.4212,0.521
svm,SVM - Linear Kernel,0.7076,0.0000,0.6908,0.8771,0.7214,0.4039,0.4839,4.111


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8469,0.0000,0.8900,0.8686,0.8790,0.6708,0.6716,3.628
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
et,Extra Trees Classifier,0.8273,0.9124,0.9223,0.8233,0.8698,0.6158,0.6254,12.915
rf,Random Forest Classifier,0.8148,0.9038,0.9065,0.8175,0.8595,0.5898,0.5974,6.766
gbc,Gradient Boosting Classifier,0.7776,0.8648,0.8837,0.7872,0.8324,0.5052,0.5136,22.113
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
ada,Ada Boost Classifier,0.7603,0.8489,0.8120,0.8089,0.8087,0.4866,0.4904,6.362
dt,Decision Tree Classifier,0.7296,0.7117,0.7834,0.7841,0.7833,0.4234,0.4241,2.384
qda,Quadratic Discriminant Analysis,0.6285,0.5084,0.9891,0.6290,0.7689,0.0205,0.0566,56.358
nb,Naive Bayes,0.6971,0.7218,0.6373,0.8397,0.7243,0.4018,0.4212,0.521


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8469,0.0000,0.8900,0.8686,0.8790,0.6708,0.6716,3.628
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
et,Extra Trees Classifier,0.8273,0.9124,0.9223,0.8233,0.8698,0.6158,0.6254,12.915
rf,Random Forest Classifier,0.8148,0.9038,0.9065,0.8175,0.8595,0.5898,0.5974,6.766
xgboost,Extreme Gradient Boosting,0.8121,0.8937,0.8577,0.8446,0.8508,0.5970,0.5977,71.451
gbc,Gradient Boosting Classifier,0.7776,0.8648,0.8837,0.7872,0.8324,0.5052,0.5136,22.113
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
ada,Ada Boost Classifier,0.7603,0.8489,0.8120,0.8089,0.8087,0.4866,0.4904,6.362
dt,Decision Tree Classifier,0.7296,0.7117,0.7834,0.7841,0.7833,0.4234,0.4241,2.384
qda,Quadratic Discriminant Analysis,0.6285,0.5084,0.9891,0.6290,0.7689,0.0205,0.0566,56.358


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8469,0.0000,0.8900,0.8686,0.8790,0.6708,0.6716,3.628
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
et,Extra Trees Classifier,0.8273,0.9124,0.9223,0.8233,0.8698,0.6158,0.6254,12.915
rf,Random Forest Classifier,0.8148,0.9038,0.9065,0.8175,0.8595,0.5898,0.5974,6.766
lightgbm,Light Gradient Boosting Machine,0.8173,0.8980,0.8665,0.8455,0.8556,0.6068,0.6079,1.566
xgboost,Extreme Gradient Boosting,0.8121,0.8937,0.8577,0.8446,0.8508,0.5970,0.5977,71.451
gbc,Gradient Boosting Classifier,0.7776,0.8648,0.8837,0.7872,0.8324,0.5052,0.5136,22.113
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
ada,Ada Boost Classifier,0.7603,0.8489,0.8120,0.8089,0.8087,0.4866,0.4904,6.362
dt,Decision Tree Classifier,0.7296,0.7117,0.7834,0.7841,0.7833,0.4234,0.4241,2.384


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8469,0.0000,0.8900,0.8686,0.8790,0.6708,0.6716,3.628
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
et,Extra Trees Classifier,0.8273,0.9124,0.9223,0.8233,0.8698,0.6158,0.6254,12.915
rf,Random Forest Classifier,0.8148,0.9038,0.9065,0.8175,0.8595,0.5898,0.5974,6.766
catboost,CatBoost Classifier,0.8184,0.8982,0.8814,0.8372,0.8585,0.6054,0.6077,91.245
lightgbm,Light Gradient Boosting Machine,0.8173,0.8980,0.8665,0.8455,0.8556,0.6068,0.6079,1.566
xgboost,Extreme Gradient Boosting,0.8121,0.8937,0.8577,0.8446,0.8508,0.5970,0.5977,71.451
gbc,Gradient Boosting Classifier,0.7776,0.8648,0.8837,0.7872,0.8324,0.5052,0.5136,22.113
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
ada,Ada Boost Classifier,0.7603,0.8489,0.8120,0.8089,0.8087,0.4866,0.4904,6.362


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8469,0.0000,0.8900,0.8686,0.8790,0.6708,0.6716,3.628
lr,Logistic Regression,0.8337,0.9139,0.9094,0.8387,0.8724,0.6348,0.6400,6.547
et,Extra Trees Classifier,0.8273,0.9124,0.9223,0.8233,0.8698,0.6158,0.6254,12.915
rf,Random Forest Classifier,0.8148,0.9038,0.9065,0.8175,0.8595,0.5898,0.5974,6.766
catboost,CatBoost Classifier,0.8184,0.8982,0.8814,0.8372,0.8585,0.6054,0.6077,91.245
lightgbm,Light Gradient Boosting Machine,0.8173,0.8980,0.8665,0.8455,0.8556,0.6068,0.6079,1.566
xgboost,Extreme Gradient Boosting,0.8121,0.8937,0.8577,0.8446,0.8508,0.5970,0.5977,71.451
gbc,Gradient Boosting Classifier,0.7776,0.8648,0.8837,0.7872,0.8324,0.5052,0.5136,22.113
knn,K Neighbors Classifier,0.7567,0.8281,0.9428,0.7406,0.8289,0.4278,0.4713,41.907
ada,Ada Boost Classifier,0.7603,0.8489,0.8120,0.8089,0.8087,0.4866,0.4904,6.362
